출처: https://blog.breezymind.com/2018/03/02/sklearn-feature_extraction-text-2/

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
np.random.seed(0)

from konlpy.tag import Okt
twitter = Okt()


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 테스트 문장
df = pd.read_csv("word2vec_wrangling.csv")

In [2]:
df.head()

,exercise_name,Content_txt
0,PT,"💯 What I try to educate my clients around, doe..."
1,검도,#20200115\n저녁 초대!\n와인잔 속에 비치는\n모든 것들이 화려한\n도심속...
2,기구필라테스,#오늘의동작\n캐딜락 동작의 완성 '행잉'\n⠀\n중력을 이용해 척추를 늘려주고\n...
3,다빈치바디보드,#mbn생생정보마당 \n#고투\n#고투GX\n#다빈치바디보드\n#생방송 #GOTOL...
4,드럼스틱,#드럼스틱 #고무팁 #테크라스틱 #전자드럼용스틱\n\n1. 전자드럼타격시 덜 시끄럽...


In [3]:
import re

def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [4]:
%time rawdata = df['Content_txt'].apply(preprocessing)

CPU times: user 789 ms, sys: 9.75 ms, total: 798 ms
Wall time: 797 ms


In [5]:
print(rawdata)

0       What I try to educate my clients around  doe...
1               저녁 초대  와인잔 속에 비치는 모든 것들이 화려한 도심속의 서울...
2      오늘의동작 캐딜락 동작의 완성  행잉    중력을 이용해 척추를 늘려주고 팔과 어...
3      mbn생생정보마당   고투  고투GX  다빈치바디보드  생방송  GOTOLIVE ...
4      드럼스틱  고무팁  테크라스틱  전자드럼용스틱     전자드럼타격시 덜 시끄럽다 ...
                            ...                        
56      오늘 날씨 어때요  따뜻한 주말 되세요   어제사진  셀카타이머인생    my ...
57        설 연휴 휴강공지            월   일 금요일과  월   일 월요일...
58        왜 바이럴 스토어를 선택해야 하나요         바이럴 스토어는      ...
59    사진을 비스듬히 찍으면 키를잃고 커보이는 상체를 얻을수 있다         팔뚝이허...
60     월 일  서울숲  시티포레스티벌  날씨  실화냐  힙레  발레핏  줌바  요가 재...
Name: Content_txt, Length: 61, dtype: object


In [6]:
vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=5,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
%time X = vectorize.fit_transform(rawdata)

# fit_transform, (sentence 5, feature 7)

print(X.toarray())

# ([[0.        , 0.40824829, 0.81649658, 0.        , 0.        , 0.        , 0.40824829],
# [0.        , 0.40824829, 0.40824829, 0.        , 0.        , 0.        , 0.81649658],
# [0.41680418, 0.        , 0.        , 0.69197025, 0.41680418, 0.41680418, 0.        ],
# [0.76944707, 0.        , 0.        , 0.63871058, 0.        , 0.        , 0.        ],
# [0.        , 0.        , 0.        , 0.8695635 , 0.34918428, 0.34918428, 0.        ]])

# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

CPU times: user 9min 5s, sys: 21.8 s, total: 9min 27s
Wall time: 7min 30s
[[0.01845768 0.         0.00949002 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.01298815 0.        ]
 [0.         0.02342017 0.         ... 0.01069141 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01089691 0.         0.02467502 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.06379826 0.        ]]


In [10]:
print(len(features))

14098


In [11]:
features[:12]

['ab',
 'abc',
 'abdominal',
 'able',
 'about',
 'abs',
 'abt',
 'ac',
 'academy',
 'account',
 'ace',
 'acrobatics']

In [8]:
# 검색 문장에서 feature를 뽑아냄. 단, 아직 띄어쓰기 모듈은 적용이 안 되어 있음
srch=[t for t in tokenizer('복근에 좋은 운동') if t in features]
print(srch)

# 검색어 예시는 다음과 같음

# 무릎 관절에 좋은 운동
# 복근 만드는 운동
# 골반 균형 맞추는 운동
# 대학교에서 동아리로 할 수 있는 운동
# 체형 교정 운동
# 척추 측만에 좋은 운동 
# 허리 디스크 운동
# 허리 통증 완화 운동
# 키 크는 운동
# 아이랑 같이 할 수 있는 운동
# 실내 운동
# 주말에 하기 좋은 운동
# 신나는 운동
# 이색적인 운동

['복근', '운동']


In [9]:
# dtm 에서 검색하고자 하는 feature만 뽑아낸다.
srch_dtm = np.asarray(X.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    vectorize.vocabulary_.get(i) for i in srch
]]

print(len(srch_dtm))
srch_dtm

61


array([[0.01171391, 0.02049993],
       [0.00513336, 0.02970128],
       [0.02089141, 0.02915134],
       [0.01628533, 0.03158734],
       [0.        , 0.01792581],
       [0.00493877, 0.02576911],
       [0.        , 0.02554627],
       [0.01857743, 0.02925842],
       [0.0208113 , 0.03383111],
       [0.        , 0.02644911],
       [0.01161136, 0.02423812],
       [0.01360429, 0.02828777],
       [0.01280858, 0.02407355],
       [0.        , 0.03262212],
       [0.00872243, 0.03306699],
       [0.01825001, 0.03178166],
       [0.01673009, 0.02967879],
       [0.        , 0.02513198],
       [0.00715791, 0.02382444],
       [0.01317643, 0.02792821],
       [0.        , 0.02494428],
       [0.        , 0.02189571],
       [0.01742788, 0.02978261],
       [0.00910236, 0.02322455],
       [0.01304624, 0.02561256],
       [0.0177172 , 0.02711924],
       [0.00413173, 0.02292213],
       [0.01154427, 0.03030849],
       [0.01899334, 0.05445109],
       [0.        , 0.07612417],
       [0.

In [10]:
score = srch_dtm.sum(axis=1)
score

array([0.03221384, 0.03483464, 0.05004274, 0.04787267, 0.01792581,
       0.03070787, 0.02554627, 0.04783585, 0.05464241, 0.02644911,
       0.03584948, 0.04189206, 0.03688213, 0.03262212, 0.04178942,
       0.05003167, 0.04640888, 0.02513198, 0.03098235, 0.04110464,
       0.02494428, 0.02189571, 0.04721049, 0.0323269 , 0.0386588 ,
       0.04483644, 0.02705386, 0.04185277, 0.07344442, 0.07612417,
       0.04895492, 0.04760584, 0.07291842, 0.03959376, 0.04886316,
       0.01846996, 0.04117015, 0.02809774, 0.05324757, 0.04278856,
       0.04032452, 0.05181382, 0.04507972, 0.04577044, 0.05379745,
       0.02106713, 0.03953615, 0.05090643, 0.03038632, 0.10587997,
       0.02805359, 0.0533093 , 0.04183989, 0.04386481, 0.05736658,
       0.09692596, 0.05408671, 0.02414853, 0.04277046, 0.05371027,
       0.04469545])

In [14]:
for i in score.argsort()[::-1]:
    if score[i] > 0.053:
        print((df['exercise_name'].iloc[i], score[i]))

('패들핏', 0.10587996957803501)
('필라테스', 0.09692596328676756)
('아쿠아테크', 0.07612417377408401)
('아쿠아바이크', 0.0734444246412008)
('에이리얼후프', 0.07291841926849542)
('플라잉필라테스', 0.05736658204813106)
('뮤직복싱', 0.05464240866113455)
('필록싱', 0.05408670617493328)
('타바타', 0.05379745411883059)
('헬스', 0.05371026884454804)
('폴댄스', 0.053309295354336136)
('점핑피트니스', 0.05324757463157929)


In [3]:
# df = pd.DataFrame([features])
# df.to_csv("features.csv")
# df

In [7]:
type(X)

scipy.sparse.csr.csr_matrix

In [24]:
# save csr matrix 
# from scipy import sparse

# sparse.save_npz("yourmatrix.npz", X)